# Valorant Project: Independence Test between Features and Target Label

<hr>  
  
### Prereq Code

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle as pkl



runs = []

with open("C:\\Users\\Binaryxx Sune\\Documents\\Programming\\personal_projects\\data\\competitive_data.pkl", "rb") as comp_data_file:
    try:
        while True:
                runs.append(pkl.load(comp_data_file))
    except:
        pass



matches = sum(runs, [])
agent = []
val_map = []
rank = []
outcome = []
team_comp = []



for match in matches:
        # get map played by each player
        for i in range(10):
            val_map.append(match["metadata"]["map"])

        # get each red player's info
        for player in match["players"]["red"]:
            # get agent selected by each player and their rank and match outcome
            agent.append(player["character"])
            rank.append(player["currenttier"])
            outcome.append(match["teams"]["red"]["has_won"])

        # get the team comp of red team
        for i in range(5):
            team_comp.append(agent[-5:])

        # get each blue player's info
        for player in match["players"]["blue"]:
            # get agent selected by each player and their rank and match outcome
            agent.append(player["character"])
            rank.append(player["currenttier"])
            outcome.append(match["teams"]["blue"]["has_won"])

        # get the team comp of blue team
        for i in range(5):
            team_comp.append(agent[-5:])


data_dict = {"agent": agent, "val_map" : val_map, "rank": rank, "win": outcome, "team_comp": team_comp}

val_data = pd.DataFrame(data_dict)
val_data = val_data[~val_data["val_map"].isin(["Breeze", "Bind"])] # filter out of cycle maps
val_data = val_data[val_data["rank"] != 0] # filter out non calibrated records


val_data

,agent,val_map,rank,win,team_comp
0,Killjoy,Haven,12,False,"[Killjoy, Jett, Phoenix, Brimstone, Skye]"
1,Jett,Haven,15,False,"[Killjoy, Jett, Phoenix, Brimstone, Skye]"
3,Brimstone,Haven,14,False,"[Killjoy, Jett, Phoenix, Brimstone, Skye]"
4,Skye,Haven,14,False,"[Killjoy, Jett, Phoenix, Brimstone, Skye]"
5,Reyna,Haven,14,True,"[Reyna, Phoenix, Fade, Killjoy, Astra]"
...,...,...,...,...,...
1165,Omen,Ascent,16,False,"[Omen, Jett, Reyna, Breach, Killjoy]"
1166,Jett,Ascent,15,False,"[Omen, Jett, Reyna, Breach, Killjoy]"
1167,Reyna,Ascent,15,False,"[Omen, Jett, Reyna, Breach, Killjoy]"
1168,Breach,Ascent,15,False,"[Omen, Jett, Reyna, Breach, Killjoy]"


In [2]:
def give_class (agent):  
    flash_duel = ["Phoenix", "Reyna", "Yoru"]
    move_duel = ["Raze", "Jett", "Neon"]
    info_ini = ["Sova", "Fade", "Breach"]
    flash_ini = ["KAY/O", "Skye"]
    dome_cont = ["Astra", "Brimstone", "Omen"]
    wall_cont = ["Harbor", "Viper"]
    sentinel = ["Cypher", "Killjoy", "Chamber", "Sage"]

    if agent in flash_duel:
        return "flash_duel"
    if agent in move_duel:
        return "move_duel"
    if agent in info_ini:
        return "info_ini"
    if agent in flash_ini:
        return "flash_ini"
    if agent in dome_cont:
        return "dome_cont"
    if agent in wall_cont:
        return "wall_cont"
    if agent in sentinel:
        return "sentinel"


val_data["class"] = val_data["agent"].apply(give_class)

val_data

,agent,val_map,rank,win,team_comp,class
0,Killjoy,Haven,12,False,"[Killjoy, Jett, Phoenix, Brimstone, Skye]",sentinel
1,Jett,Haven,15,False,"[Killjoy, Jett, Phoenix, Brimstone, Skye]",move_duel
3,Brimstone,Haven,14,False,"[Killjoy, Jett, Phoenix, Brimstone, Skye]",dome_cont
4,Skye,Haven,14,False,"[Killjoy, Jett, Phoenix, Brimstone, Skye]",flash_ini
5,Reyna,Haven,14,True,"[Reyna, Phoenix, Fade, Killjoy, Astra]",flash_duel
...,...,...,...,...,...,...
1165,Omen,Ascent,16,False,"[Omen, Jett, Reyna, Breach, Killjoy]",dome_cont
1166,Jett,Ascent,15,False,"[Omen, Jett, Reyna, Breach, Killjoy]",move_duel
1167,Reyna,Ascent,15,False,"[Omen, Jett, Reyna, Breach, Killjoy]",flash_duel
1168,Breach,Ascent,15,False,"[Omen, Jett, Reyna, Breach, Killjoy]",info_ini


In [3]:
def gen_team_comp(team_comp):
    class_comp = []
    for agent in team_comp:
        class_comp.append(give_class(agent))

    move_duel_count = []
    flash_duel_count = []
    flash_ini_count = []
    info_ini_count = []
    dome_cont_count = []
    wall_cont_count = []
    sentinel_count = []

    move_duel_count.append(class_comp.count("move_duel"))
    flash_duel_count.append(class_comp.count("flash_duel"))
    flash_ini_count.append(class_comp.count("flash_ini"))
    info_ini_count.append(class_comp.count("info_ini"))
    dome_cont_count.append(class_comp.count("dome_cont"))
    wall_cont_count.append(class_comp.count("wall_cont"))
    sentinel_count.append(class_comp.count("sentinel"))

    gen_class_comp = []
    if move_duel_count[0] != 0:
        gen_class_comp.append("md"+ str(move_duel_count[0]))
    if flash_duel_count[0] != 0:
        gen_class_comp.append("fd"+ str(flash_duel_count[0]))
    if flash_ini_count[0] != 0:
        gen_class_comp.append("fi"+ str(flash_ini_count[0]))
    if info_ini_count[0] != 0:
        gen_class_comp.append("ii"+ str(info_ini_count[0]))
    if dome_cont_count[0] != 0:
        gen_class_comp.append("dc"+ str(dome_cont_count[0]))
    if wall_cont_count[0] != 0:
        gen_class_comp.append("wc"+ str(wall_cont_count[0]))
    if sentinel_count[0] != 0:
        gen_class_comp.append("sc"+ str(sentinel_count[0]))
    
    return str(gen_class_comp)

val_data["gen_class_comp"] = val_data["team_comp"].apply(gen_team_comp)

val_data

,agent,val_map,rank,win,team_comp,class,gen_class_comp
0,Killjoy,Haven,12,False,"[Killjoy, Jett, Phoenix, Brimstone, Skye]",sentinel,"['md1', 'fd1', 'fi1', 'dc1', 'sc1']"
1,Jett,Haven,15,False,"[Killjoy, Jett, Phoenix, Brimstone, Skye]",move_duel,"['md1', 'fd1', 'fi1', 'dc1', 'sc1']"
3,Brimstone,Haven,14,False,"[Killjoy, Jett, Phoenix, Brimstone, Skye]",dome_cont,"['md1', 'fd1', 'fi1', 'dc1', 'sc1']"
4,Skye,Haven,14,False,"[Killjoy, Jett, Phoenix, Brimstone, Skye]",flash_ini,"['md1', 'fd1', 'fi1', 'dc1', 'sc1']"
5,Reyna,Haven,14,True,"[Reyna, Phoenix, Fade, Killjoy, Astra]",flash_duel,"['fd2', 'ii1', 'dc1', 'sc1']"
...,...,...,...,...,...,...,...
1165,Omen,Ascent,16,False,"[Omen, Jett, Reyna, Breach, Killjoy]",dome_cont,"['md1', 'fd1', 'ii1', 'dc1', 'sc1']"
1166,Jett,Ascent,15,False,"[Omen, Jett, Reyna, Breach, Killjoy]",move_duel,"['md1', 'fd1', 'ii1', 'dc1', 'sc1']"
1167,Reyna,Ascent,15,False,"[Omen, Jett, Reyna, Breach, Killjoy]",flash_duel,"['md1', 'fd1', 'ii1', 'dc1', 'sc1']"
1168,Breach,Ascent,15,False,"[Omen, Jett, Reyna, Breach, Killjoy]",info_ini,"['md1', 'fd1', 'ii1', 'dc1', 'sc1']"


<hr>  
  
### Testing for Independence between Features [Agent and Map]

In [4]:
# Create a Contingency Table; a table that tracks the agents selected and the map
map_agent_crosstab = pd.crosstab(val_data['agent'],
                            val_data['val_map'], 
                               margins = False)

map_agent_crosstab


val_map,Ascent,Fracture,Haven,Icebox,Lotus,Pearl,Split
agent,,,,,,,
Astra,1,2,3,2,2,5,2
Breach,3,4,3,4,5,3,4
Brimstone,13,10,7,3,9,8,8
Chamber,2,3,5,1,1,3,1
Cypher,9,5,5,4,11,6,7
Fade,8,8,12,6,9,8,5
Harbor,2,3,3,3,3,5,4
Jett,15,15,19,14,15,12,9
KAY/O,7,0,8,5,0,5,3


In [5]:
from scipy.stats import chi2_contingency

### contingency table of map and agents
map_agent_chi2 = chi2_contingency(map_agent_crosstab)

print("Test Statistic: ", map_agent_chi2[0]) 
print("P Value: ", map_agent_chi2[1]) # Probability that the data set I got is from chance given that there is no association between maps and agents; there is a correlation if below 0.1
print("Degrees of Freedom: ", map_agent_chi2[2]) 
print("Expected Frequencies: \n", map_agent_chi2[3]) 

Test Statistic:  130.91927241270182
P Value:  0.13283811486239477
Degrees of Freedom:  114
Expected Frequencies: 
 [[ 2.98134328  2.69589552  2.75932836  2.23600746  2.45802239  1.83955224
   2.02985075]
 [ 4.55970149  4.12313433  4.22014925  3.41977612  3.75932836  2.81343284
   3.10447761]
 [10.17164179  9.19776119  9.4141791   7.62873134  8.38619403  6.2761194
   6.92537313]
 [ 2.80597015  2.53731343  2.59701493  2.10447761  2.31343284  1.73134328
   1.91044776]
 [ 8.24253731  7.45335821  7.62873134  6.18190299  6.79570896  5.0858209
   5.6119403 ]
 [ 9.82089552  8.88059701  9.08955224  7.36567164  8.09701493  6.05970149
   6.68656716]
 [ 4.03358209  3.64738806  3.73320896  3.02518657  3.3255597   2.48880597
   2.74626866]
 [17.3619403  15.69962687 16.06902985 13.02145522 14.31436567 10.71268657
  11.82089552]
 [ 4.91044776  4.44029851  4.54477612  3.68283582  4.04850746  3.02985075
   3.34328358]
 [14.38059701 13.00373134 13.30970149 10.78544776 11.85634328  8.87313433
   9.7910447

In [6]:
map_agent_chi_df = pd.DataFrame(map_agent_chi2[3], columns = map_agent_crosstab.columns, index = map_agent_crosstab.index)
map_agent_chi_df

val_map,Ascent,Fracture,Haven,Icebox,Lotus,Pearl,Split
agent,,,,,,,
Astra,2.981343,2.695896,2.759328,2.236007,2.458022,1.839552,2.029851
Breach,4.559701,4.123134,4.220149,3.419776,3.759328,2.813433,3.104478
Brimstone,10.171642,9.197761,9.414179,7.628731,8.386194,6.276119,6.925373
Chamber,2.805970,2.537313,2.597015,2.104478,2.313433,1.731343,1.910448
Cypher,8.242537,7.453358,7.628731,6.181903,6.795709,5.085821,5.611940
Fade,9.820896,8.880597,9.089552,7.365672,8.097015,6.059701,6.686567
Harbor,4.033582,3.647388,3.733209,3.025187,3.325560,2.488806,2.746269
Jett,17.361940,15.699627,16.069030,13.021455,14.314366,10.712687,11.820896
KAY/O,4.910448,4.440299,4.544776,3.682836,4.048507,3.029851,3.343284


<hr>  
  
### Testing for Independence between Features [Agent and Win]

#### All Ranks

In [7]:
# Same process as above but for agents and winning
win_agent_crosstab = pd.crosstab(val_data['agent'],
                            val_data['win'], 
                               margins = False)

print(win_agent_crosstab)

print("\n")
print("\n")

### contingency table of winning and agents
win_agent_chi2 = chi2_contingency(win_agent_crosstab)

print("Test Statistic: ", win_agent_chi2[0]) 
print("P Value: ", win_agent_chi2[1]) # Probability that the data set I got is from chance given that there is no association between winning and agents; there is a correlation if below 0.1
print("Degrees of Freedom: ", win_agent_chi2[2]) 
print("Expected Frequencies: \n", win_agent_chi2[3]) 

print("\n")
print("\n")

# convert chi square chart to df
win_agent_chi_df = pd.DataFrame(win_agent_chi2[3], columns = win_agent_crosstab.columns, index = win_agent_crosstab.index)
win_agent_chi_df

win        False  True
agent                 
Astra          5    12
Breach        18     8
Brimstone     31    27
Chamber        7     9
Cypher        22    25
Fade          26    30
Harbor        13    10
Jett          52    47
KAY/O         17    11
Killjoy       44    38
Neon          11    10
Omen          38    31
Phoenix       34    31
Raze          35    36
Reyna         62    76
Sage          43    63
Skye          20    23
Sova          27    23
Viper         13    18
Yoru          10    16




Test Statistic:  18.69005243684981
P Value:  0.4768752949226811
Degrees of Freedom:  19
Expected Frequencies: 
 [[ 8.37313433  8.62686567]
 [12.80597015 13.19402985]
 [28.56716418 29.43283582]
 [ 7.88059701  8.11940299]
 [23.14925373 23.85074627]
 [27.58208955 28.41791045]
 [11.32835821 11.67164179]
 [48.76119403 50.23880597]
 [13.79104478 14.20895522]
 [40.3880597  41.6119403 ]
 [10.34328358 10.65671642]
 [33.98507463 35.01492537]
 [32.01492537 32.98507463]
 [34.97014925 36.02985075]


win,False,True
agent,,
Astra,8.373134,8.626866
Breach,12.805970,13.194030
Brimstone,28.567164,29.432836
Chamber,7.880597,8.119403
Cypher,23.149254,23.850746
Fade,27.582090,28.417910
Harbor,11.328358,11.671642
Jett,48.761194,50.238806
KAY/O,13.791045,14.208955


#### Iron 1 to Bronze 3 [Bottom 25%]

In [8]:
# Same process as above but for agents and winning
low_data = val_data[val_data["rank"].isin([1,2,3,4,5,6])]
win_agent_crosstab = pd.crosstab(low_data['agent'],
                            low_data['win'], 
                               margins = False)

print(win_agent_crosstab)

print("\n")
print("\n")

### contingency table of winning and agents
win_agent_chi2 = chi2_contingency(win_agent_crosstab)

print("Test Statistic: ", win_agent_chi2[0]) 
print("P Value: ", win_agent_chi2[1]) # Probability that the data set I got is from chance given that there is no association between winning and agents; there is a correlation if below 0.1
print("Degrees of Freedom: ", win_agent_chi2[2]) 
print("Expected Frequencies: \n", win_agent_chi2[3]) 

print("\n")
print("\n")

# convert chi square chart to df
win_agent_chi_df = pd.DataFrame(win_agent_chi2[3], columns = win_agent_crosstab.columns, index = win_agent_crosstab.index)
win_agent_chi_df

win        False  True
agent                 
Breach         0     1
Brimstone      2     0
Cypher         1     3
Fade           1     1
Harbor         7     7
KAY/O          1     0
Killjoy        1     1
Neon           3     1
Omen           1     1
Phoenix        0     1
Raze           1     0
Reyna          4     3
Sage           1     2
Skye           1     1
Sova           0     1
Viper          1     1
Yoru           0     1




Test Statistic:  10.476190476190474
P Value:  0.840606680387014
Degrees of Freedom:  16
Expected Frequencies: 
 [[0.5 0.5]
 [1.  1. ]
 [2.  2. ]
 [1.  1. ]
 [7.  7. ]
 [0.5 0.5]
 [1.  1. ]
 [2.  2. ]
 [1.  1. ]
 [0.5 0.5]
 [0.5 0.5]
 [3.5 3.5]
 [1.5 1.5]
 [1.  1. ]
 [0.5 0.5]
 [1.  1. ]
 [0.5 0.5]]






win,False,True
agent,,
Breach,0.5,0.5
Brimstone,1.0,1.0
Cypher,2.0,2.0
Fade,1.0,1.0
Harbor,7.0,7.0
KAY/O,0.5,0.5
Killjoy,1.0,1.0
Neon,2.0,2.0
Omen,1.0,1.0


#### Silver 1 to Gold 3 [25 %ile to 75 %ile]

In [9]:
# Same process as above but for agents and winning
mid_data = val_data[val_data["rank"].isin([7,8,9,10,11,12])]
win_agent_crosstab = pd.crosstab(mid_data['agent'],
                            mid_data['win'], 
                               margins = False)

print(win_agent_crosstab)

print("\n")
print("\n")

### contingency table of winning and agents
win_agent_chi2 = chi2_contingency(win_agent_crosstab)

print("Test Statistic: ", win_agent_chi2[0]) 
print("P Value: ", win_agent_chi2[1]) # Probability that the data set I got is from chance given that there is no association between winning and agents; there is a correlation if below 0.1
print("Degrees of Freedom: ", win_agent_chi2[2]) 
print("Expected Frequencies: \n", win_agent_chi2[3]) 

print("\n")
print("\n")

# convert chi square chart to df
win_agent_chi_df = pd.DataFrame(win_agent_chi2[3], columns = win_agent_crosstab.columns, index = win_agent_crosstab.index)
win_agent_chi_df

win        False  True
agent                 
Astra          1     7
Breach        12     3
Brimstone     20    14
Chamber        4     5
Cypher        16    16
Fade          17    20
Harbor         3     2
Jett          21    16
KAY/O          8     4
Killjoy       23    16
Neon           6     8
Omen          20    16
Phoenix       24    14
Raze          18    25
Reyna         35    40
Sage          31    44
Skye          10    14
Sova          13     9
Viper          5    12
Yoru           5    12




Test Statistic:  28.98482555110629
P Value:  0.0662241879163164
Degrees of Freedom:  19
Expected Frequencies: 
 [[ 3.96604414  4.03395586]
 [ 7.43633277  7.56366723]
 [16.85568761 17.14431239]
 [ 4.46179966  4.53820034]
 [15.86417657 16.13582343]
 [18.34295416 18.65704584]
 [ 2.47877759  2.52122241]
 [18.34295416 18.65704584]
 [ 5.94906621  6.05093379]
 [19.3344652  19.6655348 ]
 [ 6.94057725  7.05942275]
 [17.84719864 18.15280136]
 [18.83870968 19.16129032]
 [21.31748727 21.68251273]


win,False,True
agent,,
Astra,3.966044,4.033956
Breach,7.436333,7.563667
Brimstone,16.855688,17.144312
Chamber,4.461800,4.538200
Cypher,15.864177,16.135823
Fade,18.342954,18.657046
Harbor,2.478778,2.521222
Jett,18.342954,18.657046
KAY/O,5.949066,6.050934


#### Platinum 1 to Diamond 3 [Top 25%]

In [10]:
# Same process as above but for agents and winning
high_data = val_data[val_data["rank"].isin([13,14,15,16,17,18])]
win_agent_crosstab = pd.crosstab(high_data['agent'],
                            high_data['win'], 
                               margins = False)

print(win_agent_crosstab)

print("\n")
print("\n")

### contingency table of winning and agents
win_agent_chi2 = chi2_contingency(win_agent_crosstab)

print("Test Statistic: ", win_agent_chi2[0]) 
print("P Value: ", win_agent_chi2[1]) # Probability that the data set I got is from chance given that there is no association between winning and agents; there is a correlation if below 0.1
print("Degrees of Freedom: ", win_agent_chi2[2]) 
print("Expected Frequencies: \n", win_agent_chi2[3]) 

print("\n")
print("\n")

# convert chi square chart to df
win_agent_chi_df = pd.DataFrame(win_agent_chi2[3], columns = win_agent_crosstab.columns, index = win_agent_crosstab.index)
win_agent_chi_df

win        False  True
agent                 
Astra          4     5
Breach         5     4
Brimstone      7     9
Chamber        3     4
Cypher         5     5
Fade           6     9
Harbor         3     1
Jett          26    28
KAY/O          6     5
Killjoy       17    15
Neon           2     1
Omen          15    14
Phoenix        6    14
Raze           9     5
Reyna         20    31
Sage          10    15
Skye           9     6
Sova           9     9
Viper          4     4
Yoru           5     3




Test Statistic:  10.995163172892171
P Value:  0.924001036136701
Degrees of Freedom:  19
Expected Frequencies: 
 [[ 4.29888268  4.70111732]
 [ 4.29888268  4.70111732]
 [ 7.6424581   8.3575419 ]
 [ 3.34357542  3.65642458]
 [ 4.77653631  5.22346369]
 [ 7.16480447  7.83519553]
 [ 1.91061453  2.08938547]
 [25.79329609 28.20670391]
 [ 5.25418994  5.74581006]
 [15.2849162  16.7150838 ]
 [ 1.43296089  1.56703911]
 [13.85195531 15.14804469]
 [ 9.55307263 10.44692737]
 [ 6.68715084  7.31284916]


win,False,True
agent,,
Astra,4.298883,4.701117
Breach,4.298883,4.701117
Brimstone,7.642458,8.357542
Chamber,3.343575,3.656425
Cypher,4.776536,5.223464
Fade,7.164804,7.835196
Harbor,1.910615,2.089385
Jett,25.793296,28.206704
KAY/O,5.254190,5.745810


#### Ascendant 1 to Radiant [Top 1%]

In [11]:
# Same process as above but for agents and winning
top_data = val_data[val_data["rank"].isin([19,20,21,22,23,24,25])]
win_agent_crosstab = pd.crosstab(top_data['agent'],
                            top_data['win'], 
                               margins = False)

print(win_agent_crosstab)

print("\n")
print("\n")

### contingency table of winning and agents
win_agent_chi2 = chi2_contingency(win_agent_crosstab)

print("Test Statistic: ", win_agent_chi2[0]) 
print("P Value: ", win_agent_chi2[1]) # Probability that the data set I got is from chance given that there is no association between winning and agents; there is a correlation if below 0.1
print("Degrees of Freedom: ", win_agent_chi2[2]) 
print("Expected Frequencies: \n", win_agent_chi2[3]) 

print("\n")
print("\n")

# convert chi square chart to df
win_agent_chi_df = pd.DataFrame(win_agent_chi2[3], columns = win_agent_crosstab.columns, index = win_agent_crosstab.index)
win_agent_chi_df

win        False  True
agent                 
Breach         1     0
Brimstone      2     4
Cypher         0     1
Fade           2     0
Jett           5     3
KAY/O          2     2
Killjoy        3     6
Omen           2     0
Phoenix        4     2
Raze           7     6
Reyna          3     2
Sage           1     2
Skye           0     2
Sova           5     4
Viper          3     1




Test Statistic:  12.275927197802197
P Value:  0.5841542140328282
Degrees of Freedom:  14
Expected Frequencies: 
 [[0.53333333 0.46666667]
 [3.2        2.8       ]
 [0.53333333 0.46666667]
 [1.06666667 0.93333333]
 [4.26666667 3.73333333]
 [2.13333333 1.86666667]
 [4.8        4.2       ]
 [1.06666667 0.93333333]
 [3.2        2.8       ]
 [6.93333333 6.06666667]
 [2.66666667 2.33333333]
 [1.6        1.4       ]
 [1.06666667 0.93333333]
 [4.8        4.2       ]
 [2.13333333 1.86666667]]






win,False,True
agent,,
Breach,0.533333,0.466667
Brimstone,3.200000,2.800000
Cypher,0.533333,0.466667
Fade,1.066667,0.933333
Jett,4.266667,3.733333
KAY/O,2.133333,1.866667
Killjoy,4.800000,4.200000
Omen,1.066667,0.933333
Phoenix,3.200000,2.800000


<hr>

### Testing for Independence between Features [Class and Win]

#### All Ranks

In [12]:
# Same process as above but for agent types and winning
win_class_crosstab = pd.crosstab(val_data['class'],
                            val_data['win'], 
                               margins = False)

print(win_class_crosstab)

print("\n")
print("\n")

### contingency table of winning and agents
win_class_chi2 = chi2_contingency(win_class_crosstab)

print("Test Statistic: ", win_class_chi2[0]) 
print("P Value: ", win_class_chi2[1]) # Probability that the data set I got is from chance given that there is no association between winning and agents; there is a correlation if below 0.1
print("Degrees of Freedom: ", win_class_chi2[2]) 
print("Expected Frequencies: \n", win_class_chi2[3]) 

print("\n")
print("\n")

# convert chi square chart to df
win_class_chi_df = pd.DataFrame(win_class_chi2[3], columns = win_class_crosstab.columns, index = win_class_crosstab.index)
win_class_chi_df

win         False  True
class                  
dome_cont      74    70
flash_duel    106   123
flash_ini      37    34
info_ini       71    61
move_duel      98    93
sentinel      116   135
wall_cont      26    28




Test Statistic:  3.6626772566817514
P Value:  0.7222153114774406
Degrees of Freedom:  6
Expected Frequencies: 
 [[ 70.92537313  73.07462687]
 [112.79104478 116.20895522]
 [ 34.97014925  36.02985075]
 [ 65.01492537  66.98507463]
 [ 94.07462687  96.92537313]
 [123.62686567 127.37313433]
 [ 26.59701493  27.40298507]]






win,False,True
class,,
dome_cont,70.925373,73.074627
flash_duel,112.791045,116.208955
flash_ini,34.970149,36.029851
info_ini,65.014925,66.985075
move_duel,94.074627,96.925373
sentinel,123.626866,127.373134
wall_cont,26.597015,27.402985


#### Iron 1 to Bronze 3 [Bottom 25%]

In [13]:
# Same process as above but for agent types and winning
low_data = val_data[val_data["rank"].isin([1,2,3,4,5,6])]
win_class_crosstab = pd.crosstab(low_data['class'],
                            low_data['win'], 
                               margins = False)

print(win_class_crosstab)

print("\n")
print("\n")

### contingency table of winning and agents
win_class_chi2 = chi2_contingency(win_class_crosstab)

print("Test Statistic: ", win_class_chi2[0]) 
print("P Value: ", win_class_chi2[1]) # Probability that the data set I got is from chance given that there is no association between winning and agents; there is a correlation if below 0.1
print("Degrees of Freedom: ", win_class_chi2[2]) 
print("Expected Frequencies: \n", win_class_chi2[3]) 

print("\n")
print("\n")

# convert chi square chart to df
win_class_chi_df = pd.DataFrame(win_class_chi2[3], columns = win_class_crosstab.columns, index = win_class_crosstab.index)
win_class_chi_df

win         False  True
class                  
dome_cont       3     1
flash_duel      4     5
flash_ini       2     1
info_ini        1     3
move_duel       4     1
sentinel        3     6
wall_cont       8     8




Test Statistic:  5.2444444444444445
P Value:  0.5128652198163051
Degrees of Freedom:  6
Expected Frequencies: 
 [[2.  2. ]
 [4.5 4.5]
 [1.5 1.5]
 [2.  2. ]
 [2.5 2.5]
 [4.5 4.5]
 [8.  8. ]]






win,False,True
class,,
dome_cont,2.0,2.0
flash_duel,4.5,4.5
flash_ini,1.5,1.5
info_ini,2.0,2.0
move_duel,2.5,2.5
sentinel,4.5,4.5
wall_cont,8.0,8.0


#### Silver 1 to Gold 3 [25 %ile to 75 %ile]

In [14]:
mid_data = val_data[val_data["rank"].isin([7,8,9,10,11,12])]
win_class_crosstab = pd.crosstab(mid_data['class'],
                            mid_data['win'], 
                               margins = False)

print(win_class_crosstab)

print("\n")
print("\n")

### contingency table of winning and agents
win_class_chi2 = chi2_contingency(win_class_crosstab)

print("Test Statistic: ", win_class_chi2[0]) 
print("P Value: ", win_class_chi2[1]) # Probability that the data set I got is from chance given that there is no association between winning and agents; there is a correlation if below 0.1
print("Degrees of Freedom: ", win_class_chi2[2]) 
print("Expected Frequencies: \n", win_class_chi2[3]) 

print("\n")
print("\n")

# convert chi square chart to df
win_class_chi_df = pd.DataFrame(win_class_chi2[3], columns = win_class_crosstab.columns, index = win_class_crosstab.index)
win_class_chi_df

win         False  True
class                  
dome_cont      41    37
flash_duel     64    66
flash_ini      18    18
info_ini       42    32
move_duel      45    49
sentinel       74    81
wall_cont       8    14




Test Statistic:  3.6677737090965836
P Value:  0.7215306818085772
Degrees of Freedom:  6
Expected Frequencies: 
 [[38.66893039 39.33106961]
 [64.44821732 65.55178268]
 [17.84719864 18.15280136]
 [36.68590832 37.31409168]
 [46.60101868 47.39898132]
 [76.84210526 78.15789474]
 [10.90662139 11.09337861]]






win,False,True
class,,
dome_cont,38.668930,39.331070
flash_duel,64.448217,65.551783
flash_ini,17.847199,18.152801
info_ini,36.685908,37.314092
move_duel,46.601019,47.398981
sentinel,76.842105,78.157895
wall_cont,10.906621,11.093379


#### Platinum 1 to Diamond 3 [Top 25%]

In [15]:
high_data = val_data[val_data["rank"].isin([13,14,15,16,17,18])]
win_class_crosstab = pd.crosstab(high_data['class'],
                            high_data['win'], 
                               margins = False)

print(win_class_crosstab)

print("\n")
print("\n")

### contingency table of winning and agents
win_class_chi2 = chi2_contingency(win_class_crosstab)

print("Test Statistic: ", win_class_chi2[0]) 
print("P Value: ", win_class_chi2[1]) # Probability that the data set I got is from chance given that there is no association between winning and agents; there is a correlation if below 0.1
print("Degrees of Freedom: ", win_class_chi2[2]) 
print("Expected Frequencies: \n", win_class_chi2[3]) 

print("\n")
print("\n")

# convert chi square chart to df
win_class_chi_df = pd.DataFrame(win_class_chi2[3], columns = win_class_crosstab.columns, index = win_class_crosstab.index)
win_class_chi_df

win         False  True
class                  
dome_cont      26    28
flash_duel     31    48
flash_ini      15    11
info_ini       20    22
move_duel      37    34
sentinel       35    39
wall_cont       7     5




Test Statistic:  4.4129655842595295
P Value:  0.620975947652155
Degrees of Freedom:  6
Expected Frequencies: 
 [[25.79329609 28.20670391]
 [37.73463687 41.26536313]
 [12.41899441 13.58100559]
 [20.06145251 21.93854749]
 [33.91340782 37.08659218]
 [35.34636872 38.65363128]
 [ 5.73184358  6.26815642]]






win,False,True
class,,
dome_cont,25.793296,28.206704
flash_duel,37.734637,41.265363
flash_ini,12.418994,13.581006
info_ini,20.061453,21.938547
move_duel,33.913408,37.086592
sentinel,35.346369,38.653631
wall_cont,5.731844,6.268156


#### Ascendant 1 to Radiant [Top 1%]

In [16]:
high_data = val_data[val_data["rank"].isin([13,14,15,16,17,18])]
win_class_crosstab = pd.crosstab(high_data['class'],
                            high_data['win'], 
                               margins = False)

print(win_class_crosstab)

print("\n")
print("\n")

### contingency table of winning and agents
win_class_chi2 = chi2_contingency(win_class_crosstab)

print("Test Statistic: ", win_class_chi2[0]) 
print("P Value: ", win_class_chi2[1]) # Probability that the data set I got is from chance given that there is no association between winning and agents; there is a correlation if below 0.1
print("Degrees of Freedom: ", win_class_chi2[2]) 
print("Expected Frequencies: \n", win_class_chi2[3]) 

print("\n")
print("\n")

# convert chi square chart to df
win_class_chi_df = pd.DataFrame(win_class_chi2[3], columns = win_class_crosstab.columns, index = win_class_crosstab.index)
win_class_chi_df

win         False  True
class                  
dome_cont      26    28
flash_duel     31    48
flash_ini      15    11
info_ini       20    22
move_duel      37    34
sentinel       35    39
wall_cont       7     5




Test Statistic:  4.4129655842595295
P Value:  0.620975947652155
Degrees of Freedom:  6
Expected Frequencies: 
 [[25.79329609 28.20670391]
 [37.73463687 41.26536313]
 [12.41899441 13.58100559]
 [20.06145251 21.93854749]
 [33.91340782 37.08659218]
 [35.34636872 38.65363128]
 [ 5.73184358  6.26815642]]






win,False,True
class,,
dome_cont,25.793296,28.206704
flash_duel,37.734637,41.265363
flash_ini,12.418994,13.581006
info_ini,20.061453,21.938547
move_duel,33.913408,37.086592
sentinel,35.346369,38.653631
wall_cont,5.731844,6.268156


<hr>

### Testing Independence between Features [Team Comp and Win]

#### Prereq Code

In [17]:
def dom_comp(comp):
    top10 = [['md1', 'fd1', 'ii1', 'dc1', 'sc1'], ['md1', 'fd1', 'fi1', 'dc1', 'sc1'], ['md1', 'fd1', 'ii1', 'wc1', 'sc1'], ['fd2', 'ii1', 'dc1', 'sc1'], ['md1', 'fd1', 'fi1', 'wc1', 'sc1'],
    ['fd1', 'ii1', 'dc1', 'sc2'], ['md2', 'ii1', 'dc1', 'sc1'], ['fd2', 'fi1', 'dc1', 'sc1'], ['md1', 'fd1', 'dc1', 'sc2']]
    top10 = [str(i) for i in top10]
    
    if comp in top10:
        return comp
    else:
        return "Non Dominant"

val_data["gen_class_comp"] = val_data["gen_class_comp"].apply(dom_comp)
val_data

,agent,val_map,rank,win,team_comp,class,gen_class_comp
0,Killjoy,Haven,12,False,"[Killjoy, Jett, Phoenix, Brimstone, Skye]",sentinel,"['md1', 'fd1', 'fi1', 'dc1', 'sc1']"
1,Jett,Haven,15,False,"[Killjoy, Jett, Phoenix, Brimstone, Skye]",move_duel,"['md1', 'fd1', 'fi1', 'dc1', 'sc1']"
3,Brimstone,Haven,14,False,"[Killjoy, Jett, Phoenix, Brimstone, Skye]",dome_cont,"['md1', 'fd1', 'fi1', 'dc1', 'sc1']"
4,Skye,Haven,14,False,"[Killjoy, Jett, Phoenix, Brimstone, Skye]",flash_ini,"['md1', 'fd1', 'fi1', 'dc1', 'sc1']"
5,Reyna,Haven,14,True,"[Reyna, Phoenix, Fade, Killjoy, Astra]",flash_duel,"['fd2', 'ii1', 'dc1', 'sc1']"
...,...,...,...,...,...,...,...
1165,Omen,Ascent,16,False,"[Omen, Jett, Reyna, Breach, Killjoy]",dome_cont,"['md1', 'fd1', 'ii1', 'dc1', 'sc1']"
1166,Jett,Ascent,15,False,"[Omen, Jett, Reyna, Breach, Killjoy]",move_duel,"['md1', 'fd1', 'ii1', 'dc1', 'sc1']"
1167,Reyna,Ascent,15,False,"[Omen, Jett, Reyna, Breach, Killjoy]",flash_duel,"['md1', 'fd1', 'ii1', 'dc1', 'sc1']"
1168,Breach,Ascent,15,False,"[Omen, Jett, Reyna, Breach, Killjoy]",info_ini,"['md1', 'fd1', 'ii1', 'dc1', 'sc1']"


#### All Ranks

In [18]:
win_class_crosstab = pd.crosstab(val_data['gen_class_comp'],
                            val_data['win'], 
                               margins = False)

print(win_class_crosstab)

print("\n")
print("\n")

### contingency table of winning and agents
win_class_chi2 = chi2_contingency(win_class_crosstab)

print("Test Statistic: ", win_class_chi2[0]) 
print("P Value: ", win_class_chi2[1]) # Probability that the data set I got is from chance given that there is no association between winning and agents; there is a correlation if below 0.1
print("Degrees of Freedom: ", win_class_chi2[2]) 
print("Expected Frequencies: \n", win_class_chi2[3]) 

print("\n")
print("\n")

# convert chi square chart to df
win_class_chi_df = pd.DataFrame(win_class_chi2[3], columns = win_class_crosstab.columns, index = win_class_crosstab.index)
win_class_chi_df

win                                  False  True
gen_class_comp                                  
Non Dominant                           262   261
['fd1', 'ii1', 'dc1', 'sc2']             5    33
['fd2', 'fi1', 'dc1', 'sc1']            23    10
['fd2', 'ii1', 'dc1', 'sc1']            37    34
['md1', 'fd1', 'dc1', 'sc2']            18    15
['md1', 'fd1', 'fi1', 'dc1', 'sc1']     35    49
['md1', 'fd1', 'fi1', 'wc1', 'sc1']     14    27
['md1', 'fd1', 'ii1', 'dc1', 'sc1']     71    61
['md1', 'fd1', 'ii1', 'wc1', 'sc1']     48    34
['md2', 'ii1', 'dc1', 'sc1']            15    20




Test Statistic:  36.24084816401505
P Value:  3.5928528339092574e-05
Degrees of Freedom:  9
Expected Frequencies: 
 [[257.59701493 265.40298507]
 [ 18.71641791  19.28358209]
 [ 16.25373134  16.74626866]
 [ 34.97014925  36.02985075]
 [ 16.25373134  16.74626866]
 [ 41.37313433  42.62686567]
 [ 20.19402985  20.80597015]
 [ 65.01492537  66.98507463]
 [ 40.3880597   41.6119403 ]
 [ 17.23880597  17.76119403]]




win,False,True
gen_class_comp,,
Non Dominant,257.597015,265.402985
"['fd1', 'ii1', 'dc1', 'sc2']",18.716418,19.283582
"['fd2', 'fi1', 'dc1', 'sc1']",16.253731,16.746269
"['fd2', 'ii1', 'dc1', 'sc1']",34.970149,36.029851
"['md1', 'fd1', 'dc1', 'sc2']",16.253731,16.746269
"['md1', 'fd1', 'fi1', 'dc1', 'sc1']",41.373134,42.626866
"['md1', 'fd1', 'fi1', 'wc1', 'sc1']",20.194030,20.805970
"['md1', 'fd1', 'ii1', 'dc1', 'sc1']",65.014925,66.985075
"['md1', 'fd1', 'ii1', 'wc1', 'sc1']",40.388060,41.611940


#### Iron 1 to Bronze 3 [Bottom 25%]

In [19]:
# Same process as above but for agents and winning
low_data = val_data[val_data["rank"].isin([1,2,3,4,5,6])]
win_agent_crosstab = pd.crosstab(low_data['gen_class_comp'],
                            low_data['win'], 
                               margins = False)

print(win_agent_crosstab)

print("\n")
print("\n")

### contingency table of winning and agents
win_agent_chi2 = chi2_contingency(win_agent_crosstab)

print("Test Statistic: ", win_agent_chi2[0]) 
print("P Value: ", win_agent_chi2[1]) # Probability that the data set I got is from chance given that there is no association between winning and agents; there is a correlation if below 0.1
print("Degrees of Freedom: ", win_agent_chi2[2]) 
print("Expected Frequencies: \n", win_agent_chi2[3]) 

print("\n")
print("\n")

# convert chi square chart to df
win_agent_chi_df = pd.DataFrame(win_agent_chi2[3], columns = win_agent_crosstab.columns, index = win_agent_crosstab.index)
win_agent_chi_df

win                                  False  True
gen_class_comp                                  
Non Dominant                            17    17
['fd2', 'ii1', 'dc1', 'sc1']             0     2
['md1', 'fd1', 'dc1', 'sc2']             1     0
['md1', 'fd1', 'fi1', 'dc1', 'sc1']      2     1
['md1', 'fd1', 'fi1', 'wc1', 'sc1']      0     5
['md1', 'fd1', 'ii1', 'wc1', 'sc1']      5     0




Test Statistic:  13.333333333333332
P Value:  0.020447481945815256
Degrees of Freedom:  5
Expected Frequencies: 
 [[17.  17. ]
 [ 1.   1. ]
 [ 0.5  0.5]
 [ 1.5  1.5]
 [ 2.5  2.5]
 [ 2.5  2.5]]






win,False,True
gen_class_comp,,
Non Dominant,17.0,17.0
"['fd2', 'ii1', 'dc1', 'sc1']",1.0,1.0
"['md1', 'fd1', 'dc1', 'sc2']",0.5,0.5
"['md1', 'fd1', 'fi1', 'dc1', 'sc1']",1.5,1.5
"['md1', 'fd1', 'fi1', 'wc1', 'sc1']",2.5,2.5
"['md1', 'fd1', 'ii1', 'wc1', 'sc1']",2.5,2.5


#### Silver 1 to Gold 3 [25 %ile to 75 %ile]

In [20]:
mid_data = val_data[val_data["rank"].isin([7,8,9,10,11,12])]
win_class_crosstab = pd.crosstab(mid_data['gen_class_comp'],
                            mid_data['win'], 
                               margins = False)

print(win_class_crosstab)

print("\n")
print("\n")

### contingency table of winning and agents
win_class_chi2 = chi2_contingency(win_class_crosstab)

print("Test Statistic: ", win_class_chi2[0]) 
print("P Value: ", win_class_chi2[1]) # Probability that the data set I got is from chance given that there is no association between winning and agents; there is a correlation if below 0.1
print("Degrees of Freedom: ", win_class_chi2[2]) 
print("Expected Frequencies: \n", win_class_chi2[3]) 

print("\n")
print("\n")

# convert chi square chart to df
win_class_chi_df = pd.DataFrame(win_class_chi2[3], columns = win_class_crosstab.columns, index = win_class_crosstab.index)
win_class_chi_df

win                                  False  True
gen_class_comp                                  
Non Dominant                           150   148
['fd1', 'ii1', 'dc1', 'sc2']             5    12
['fd2', 'fi1', 'dc1', 'sc1']            11     5
['fd2', 'ii1', 'dc1', 'sc1']            22    19
['md1', 'fd1', 'dc1', 'sc2']            12    11
['md1', 'fd1', 'fi1', 'dc1', 'sc1']     20    25
['md1', 'fd1', 'fi1', 'wc1', 'sc1']      5    15
['md1', 'fd1', 'ii1', 'dc1', 'sc1']     35    34
['md1', 'fd1', 'ii1', 'wc1', 'sc1']     23    21
['md2', 'ii1', 'dc1', 'sc1']             9     7




Test Statistic:  11.278091521963086
P Value:  0.2571278016995273
Degrees of Freedom:  9
Expected Frequencies: 
 [[147.73514431 150.26485569]
 [  8.4278438    8.5721562 ]
 [  7.93208829   8.06791171]
 [ 20.32597623  20.67402377]
 [ 11.40237691  11.59762309]
 [ 22.3089983   22.6910017 ]
 [  9.91511036  10.08488964]
 [ 34.20713073  34.79286927]
 [ 21.81324278  22.18675722]
 [  7.93208829   8.06791171]]






win,False,True
gen_class_comp,,
Non Dominant,147.735144,150.264856
"['fd1', 'ii1', 'dc1', 'sc2']",8.427844,8.572156
"['fd2', 'fi1', 'dc1', 'sc1']",7.932088,8.067912
"['fd2', 'ii1', 'dc1', 'sc1']",20.325976,20.674024
"['md1', 'fd1', 'dc1', 'sc2']",11.402377,11.597623
"['md1', 'fd1', 'fi1', 'dc1', 'sc1']",22.308998,22.691002
"['md1', 'fd1', 'fi1', 'wc1', 'sc1']",9.915110,10.084890
"['md1', 'fd1', 'ii1', 'dc1', 'sc1']",34.207131,34.792869
"['md1', 'fd1', 'ii1', 'wc1', 'sc1']",21.813243,22.186757


#### Platinum 1 to Diamond 3 [Top 25%]

In [21]:
high_data = val_data[val_data["rank"].isin([13,14,15,16,17,18])]
win_class_crosstab = pd.crosstab(high_data['gen_class_comp'],
                            high_data['win'], 
                               margins = False)

print(win_class_crosstab)

print("\n")
print("\n")

### contingency table of winning and agents
win_class_chi2 = chi2_contingency(win_class_crosstab)

print("Test Statistic: ", win_class_chi2[0]) 
print("P Value: ", win_class_chi2[1]) # Probability that the data set I got is from chance given that there is no association between winning and agents; there is a correlation if below 0.1
print("Degrees of Freedom: ", win_class_chi2[2]) 
print("Expected Frequencies: \n", win_class_chi2[3]) 

print("\n")
print("\n")

# convert chi square chart to df
win_class_chi_df = pd.DataFrame(win_class_chi2[3], columns = win_class_crosstab.columns, index = win_class_crosstab.index)
win_class_chi_df

win                                  False  True
gen_class_comp                                  
Non Dominant                            84    78
['fd1', 'ii1', 'dc1', 'sc2']             0    21
['fd2', 'fi1', 'dc1', 'sc1']             8     4
['fd2', 'ii1', 'dc1', 'sc1']            14    13
['md1', 'fd1', 'dc1', 'sc2']             5     4
['md1', 'fd1', 'fi1', 'dc1', 'sc1']     13    17
['md1', 'fd1', 'fi1', 'wc1', 'sc1']      8     5
['md1', 'fd1', 'ii1', 'dc1', 'sc1']     27    24
['md1', 'fd1', 'ii1', 'wc1', 'sc1']      8    13
['md2', 'ii1', 'dc1', 'sc1']             4     8




Test Statistic:  25.966407403538867
P Value:  0.0020690038522384727
Degrees of Freedom:  9
Expected Frequencies: 
 [[77.37988827 84.62011173]
 [10.03072626 10.96927374]
 [ 5.73184358  6.26815642]
 [12.89664804 14.10335196]
 [ 4.29888268  4.70111732]
 [14.32960894 15.67039106]
 [ 6.20949721  6.79050279]
 [24.3603352  26.6396648 ]
 [10.03072626 10.96927374]
 [ 5.73184358  6.26815642]]






win,False,True
gen_class_comp,,
Non Dominant,77.379888,84.620112
"['fd1', 'ii1', 'dc1', 'sc2']",10.030726,10.969274
"['fd2', 'fi1', 'dc1', 'sc1']",5.731844,6.268156
"['fd2', 'ii1', 'dc1', 'sc1']",12.896648,14.103352
"['md1', 'fd1', 'dc1', 'sc2']",4.298883,4.701117
"['md1', 'fd1', 'fi1', 'dc1', 'sc1']",14.329609,15.670391
"['md1', 'fd1', 'fi1', 'wc1', 'sc1']",6.209497,6.790503
"['md1', 'fd1', 'ii1', 'dc1', 'sc1']",24.360335,26.639665
"['md1', 'fd1', 'ii1', 'wc1', 'sc1']",10.030726,10.969274


#### Ascendant 1 to Radiant [Top 1%]

In [22]:
# Same process as above but for agents and winning
top_data = val_data[val_data["rank"].isin([19,20,21,22,23,24,25])]
win_agent_crosstab = pd.crosstab(top_data['gen_class_comp'],
                            top_data['win'], 
                               margins = False)

print(win_agent_crosstab)

print("\n")
print("\n")

### contingency table of winning and agents
win_agent_chi2 = chi2_contingency(win_agent_crosstab)

print("Test Statistic: ", win_agent_chi2[0]) 
print("P Value: ", win_agent_chi2[1]) # Probability that the data set I got is from chance given that there is no association between winning and agents; there is a correlation if below 0.1
print("Degrees of Freedom: ", win_agent_chi2[2]) 
print("Expected Frequencies: \n", win_agent_chi2[3]) 

print("\n")
print("\n")

# convert chi square chart to df
win_agent_chi_df = pd.DataFrame(win_agent_chi2[3], columns = win_agent_crosstab.columns, index = win_agent_crosstab.index)
win_agent_chi_df

win                                  False  True
gen_class_comp                                  
Non Dominant                            11    18
['fd2', 'fi1', 'dc1', 'sc1']             4     1
['fd2', 'ii1', 'dc1', 'sc1']             1     0
['md1', 'fd1', 'fi1', 'dc1', 'sc1']      0     6
['md1', 'fd1', 'fi1', 'wc1', 'sc1']      1     2
['md1', 'fd1', 'ii1', 'dc1', 'sc1']      9     3
['md1', 'fd1', 'ii1', 'wc1', 'sc1']     12     0
['md2', 'ii1', 'dc1', 'sc1']             2     5




Test Statistic:  26.894902357494722
P Value:  0.0003481111174912069
Degrees of Freedom:  7
Expected Frequencies: 
 [[15.46666667 13.53333333]
 [ 2.66666667  2.33333333]
 [ 0.53333333  0.46666667]
 [ 3.2         2.8       ]
 [ 1.6         1.4       ]
 [ 6.4         5.6       ]
 [ 6.4         5.6       ]
 [ 3.73333333  3.26666667]]






win,False,True
gen_class_comp,,
Non Dominant,15.466667,13.533333
"['fd2', 'fi1', 'dc1', 'sc1']",2.666667,2.333333
"['fd2', 'ii1', 'dc1', 'sc1']",0.533333,0.466667
"['md1', 'fd1', 'fi1', 'dc1', 'sc1']",3.200000,2.800000
"['md1', 'fd1', 'fi1', 'wc1', 'sc1']",1.600000,1.400000
"['md1', 'fd1', 'ii1', 'dc1', 'sc1']",6.400000,5.600000
"['md1', 'fd1', 'ii1', 'wc1', 'sc1']",6.400000,5.600000
"['md2', 'ii1', 'dc1', 'sc1']",3.733333,3.266667
